In [ ]:
from pyqe import *

total_patients_query = Query('ADHD Phenotype') # Always begin your script by creating Query object

total_patients_query.get_study_list()
total_patients_query.set_study('62ce796d-f38a-4dab-95b9-00d202ece4fe') # any STUDY_ID from above list
total_patients_query.set_study_config('14356623-1766-4acb-9e16-9285d47bb71c') # any config id from above list

In [ ]:
from pyqe import *

# Patients who are 4 years old and above
patient_data = Person.Patient()
constraint_age_greater_than_4_years = Constraint()
constraint_age_greater_than_4_years.add(Expression(ComparisonOperator.MORE_THAN_EQUAL, 4))
patient_data.add_age([constraint_age_greater_than_4_years])

In [ ]:
# Patients who are diagnosed with ADHD conditions
adhd_conditionOcc = Interactions.ConditionOccurrence("ADHD conditions Interaction")
adhd_condition_concepts = ConceptSet(
                           'Conditions',
                            Domain.CONDITION, 
                           ['406506008', '192132008', '192131001']) 
adhd_conditionOcc.add_concept_set(adhd_condition_concepts)

# Add Patients who are prescribed with ADHD medications
adhd_drug_concepts  = ConceptSet('ADHD Medications',
                            Domain.DRUG, 
                           ['1091497', '2599', '725', '40114', '308979', '308976', '308973', 
                            '2598', '4493', '36437', '35636', '32937', '10737',
                            '310384', '313990','310385', '313989', '312938', 
                            '312940', '312941', '374185'])

adhd_drugExposure = Interactions.DrugExposure("ADHD Medications Interaction")
adhd_drugExposure.add_concept_set(adhd_drug_concepts)


# Exclusion of Certain Conditions like Dementia
exclude_other_conditionOcc = Interactions.ConditionOccurrence("Other Disorders", CardType.EXCLUDED)
exclude_condition_concepts = ConceptSet('Conditions',
                            Domain.CONDITION, 
                           ['191449005', '397923000', '31297008', '18393005', 
                            '50705009', '5507002', '86765009', '110359009', 
                            '40700009', '31216003', '61152003'])
exclude_other_conditionOcc.add_concept_set(exclude_condition_concepts)

In [ ]:
# Combine the criteria for demographic data, ADHD conditions & medications
ADHD_group = CriteriaGroup(
    MatchCriteria.ALL, [patient_data, adhd_conditionOcc, exclude_other_conditionOcc])

# Add Exclusive to the final Criteria Group
exclusive_group = CriteriaGroup(
    MatchCriteria.ANY, [adhd_drugExposure])

ADHD_group.add_exclusive_group(exclusive_group)

# Add criteria group into query
total_patients_query.add_criteria_group(ADHD_group)

In [ ]:
# Generate the request
request = total_patients_query.get_patient_count_filter()

# Get the result from the request
patient_count = Result().get_patient_count(request)
print(f'\nTotal number of patients for ADHD Phenotype: {patient_count}')

In [ ]:
# Generate Request for Condition Occurrence Dataframe cohort
request_co_df = total_patients_query.get_dataframe_cohort()

# Get Condition Occurrence Dataframe
condition_occurrence_dataframe = Result().download_dataframe(request_co_df)

In [ ]:
# Peek Into the Conditions Data
condition_occurrence_dataframe_simple = condition_occurrence_dataframe[['pid', 'conditionname', 'condconceptcode', 'conditionstatus', 'startdate', 'enddate']]
condition_occurrence_dataframe_simple.head(15)

In [ ]:
import matplotlib.pyplot as plt

# Group Data by Conditions
co_mini_df = condition_occurrence_dataframe_simple[['conditionname', 'pid']]
co_group_df = co_mini_df.groupby(['conditionname']).count()
co_group_df.rename(columns = {'pid':'count'}, inplace = True)

# Sort & Pick the top 10 conditions
co_group_df.sort_values(by=['count'], inplace=True, ascending=False)
co_group_df_top10 = co_group_df.head(10)
plt.figure(figsize=(16,8))

# plot pie chart
ax1 = plt.subplot(121, aspect='equal')
co_group_df_top10.plot(kind='pie', y = 'count', ax=ax1, autopct='%1.1f%%', 
 startangle=90, shadow=False, legend = False, fontsize=14)

plt.title('ADHD & Other Comorbidities', fontdict={'fontsize':20})
plt.axis('off')
plt.show()

In [ ]:
# Generate Request for Dataframe cohort
request_df = total_patients_query.get_dataframe_cohort()

# Get Patient Dataframe
patient_dataframe = Result().download_dataframe(request_df)

In [ ]:
# Peek Into the Patient Demographics Data
df1 = patient_dataframe[['pid', 'age', 'gender', 'ethnicity', 'race', 'state']]
df1.head(20)

In [ ]:
# Plot bar chart based on Gender & Race
barh_table = pd.crosstab(patient_dataframe.race, patient_dataframe.gender)
ax = barh_table.plot.barh(figsize=(15,15), title='Group by Gender & Race Categories')
ax.yaxis.set_tick_params(labelsize='xx-large')
ax.title.set_size(25)

In [ ]:
import pandas as pd

# Null Hypotheses: Gender & Race are independent of each other regarding ADHD condition
significance = 0.05

# Two Way Table Observed Values
observed_table = pd.crosstab(patient_dataframe.race, patient_dataframe.gender, margins=True, margins_name="Total")
observed_table.rename(columns = {'Total':'Row Total'}, inplace = True)
observed_table.rename(index = {'Total':'Col Total'}, inplace = True)
observed_table

In [ ]:
from scipy.stats import chi2_contingency 
import numpy as np
chisqt = pd.crosstab(patient_dataframe.race, patient_dataframe.gender, margins=True)
value = np.array([chisqt.iloc[[0,1,2,3,4,5,6,7,8,9,10,11],[0, 1]].values,
                 ])
stat, p, dof = chi2_contingency(value)[0:3]
print('Chi-square Analysis:')
print('p-value=%.3f, degrees of freedom=%i, statistics=%.3f' % (p, dof, stat))

In [ ]:
# Check p-value
print('significance=%.3f, p-value=%.3f' % (significance, p))
if p <= significance:
	print('Result: Dependent - Reject Null Hypothesis (Race and Gender are dependent)')
else:
	print('Result: Independent - Accept Null Hypothesis (Race and Gender are independent)')

In [ ]:
# Check test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Result: Dependent - Reject Null Hypothesis (Race and Gender are dependent)')
else:
	print('Result: Independent - Accept Null Hypothesis (Race and Gender are independent)')